# Warfarin PopPk modeling and simulation

In [1]:
from mas.model import *


class Warfarin(pk.EvOneCmtLinear):

    def __init__(self) -> None:
        super().__init__()
        self.theta_V = theta(7.81, bounds=(0, None))
        self.theta_Cl = theta(0.134, bounds=(0, None))
        self.theta_ka = theta(0.571, bounds=(0, None))
        self.theta_alag = theta(0.823, bounds=(0, None))

        self.eta_Cl = omega(0.049)
        self.eta_V = omega(0.0802)
        self.eta_ka = omega(0.047)
        self.eta_alag = omega(0.156)

        self.eps_prop = sigma(0.0104)
        self.eps_add = sigma(0.554)

    def pred(self) -> Expression:
        cl = self.theta_Cl * exp(self.eta_Cl)
        v = self.theta_V * exp(self.eta_V)
        ka = self.theta_ka * exp(self.eta_ka)
        alag = self.theta_alag * exp(self.eta_alag)

        amts = self.trans2(cl=cl, v=v, ka=ka, alag1=alag).A
        ipred = amts["central"] / v
        # ipred = subroutine.F
        y = ipred * (1 + self.eps_prop) + self.eps_add

        return y


mod = Warfarin()

In [2]:
import pandas as pd

model_data = ModelData(pd.read_csv(
    'https://clinpharmacol.fmhs.auckland.ac.nz/docs/warfarin.csv', 
    na_values=["."], 
    comment="#", 
    header=None, 
    names=["ID", "time", "wt", "age", "sex", "amt", "rate", "dvid", "dv", "mdv"]
))
model_data = model_data.drop(columns=["rate"])
model_data = model_data[model_data["dvid"] != 2].reset_index(drop=True)
model_data.head(20)

    ID   time    wt  age  sex    amt  dvid    dv  mdv
0    0    0.0  66.7   50    1  100.0     0   NaN    1
1    0    0.5  66.7   50    1    NaN     1   0.0    0
2    0    1.0  66.7   50    1    NaN     1   1.9    0
3    0    2.0  66.7   50    1    NaN     1   3.3    0
4    0    3.0  66.7   50    1    NaN     1   6.6    0
5    0    6.0  66.7   50    1    NaN     1   9.1    0
6    0    9.0  66.7   50    1    NaN     1  10.8    0
7    0   12.0  66.7   50    1    NaN     1   8.6    0
8    0   24.0  66.7   50    1    NaN     1   5.6    0
9    0   36.0  66.7   50    1    NaN     1   4.0    0
10   0   48.0  66.7   50    1    NaN     1   2.7    0
11   0   72.0  66.7   50    1    NaN     1   0.8    0
12   0   96.0  66.7   50    1    NaN     1   NaN    1
13   0  120.0  66.7   50    1    NaN     1   NaN    1
14   1    0.0  66.7   50    1  100.0     0   NaN    1
15   1   24.0  66.7   50    1    NaN     1   9.2    0
16   1   36.0  66.7   50    1    NaN     1   8.5    0
17   1   48.0  66.7   50    

In [3]:
pop_model = PopModel(mod=mod, data=model_data)

# fit_result = pop_model.fit(FOCEi(minimize_method='nm', log_level="no"))
fit_result = pop_model.fit(FOCEi(minimize_method='bobyqa', print_type='progress'))
# fit_result.compute_se()

➡️ Since build cache already exists, skip compile
✈️ First Order Conditional Estimation

* maxiter = 2000
* xtol = 0.000001
* ftol = 0.000001
* lower_bounds = 
* upper_bounds = 
* log_level = 1
* print = 1
* print_type = 1
Using Bobyqa...
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 50
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 100
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 150
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 200
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 250
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 300
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 350
|| ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || ===== | ===== || 400
|| ===== | ===== || ===== 

In [4]:
print(fit_result)

================= Summary =================

*** Fit Summary ***
| Final Objective Value   | Converged   | Total CPU Elapsed Time   | Total Iterations   |
|-------------------------|-------------|--------------------------|--------------------|
| 216.802                 | True ✅     | 0:00:02.33               | 458                |

*** Parameter Estimation ***
| Parameter       | Estimate   | RSE(%)   | Shrinkage(%)   |
|-----------------|------------|----------|----------------|
| **Theta**       |            |          |                |
| theta_V         | 7.969      | -        | -              |
| theta_Cl        | 0.132      | -        | -              |
| theta_ka        | 1.408      | -        | -              |
| theta_alag      | 0.882      | -        | -              |
| **Omega[SD]**   |            |          |                |
| Omega(eta_Cl)   | 0.291      | -        | 1.5            |
| Omega(eta_V)    | 0.218      | -        | 5.3            |
| Omega(eta_ka)   | 0.795 

In [5]:
trace_plot = fit_result.plot_iteration_trace()
trace_plot.show()

In [6]:
gof_plot = fit_result.plot_goodness_of_fit()
gof_plot.show()

In [7]:
ind_plot = fit_result.plot_individual_fit(cols=3)
ind_plot.show()